Разработаем систему классификации отзывов к фильмам на положительные и отрицательные. Такая система может быть полезна, например, компаниям, которые анализируют отзывы в социальных сетях, и решают, стоит ли снимать продолжение к фильму.

Это задача на работу с естественным языком. Несмотря на то, что применяемые техники довольно простые, они позволят проиллюстрировать как задачу классификации, так и несколько основных идей из области анализа текстов. 

### Модель «Мешок Слов»

Одной из простейших идей при работе с текстами является модель «мешка слов» (bag of the words, bow). Идея заключается в том, что порядок слов для тематики не важен, а важны только сами слова. 

Частота встречающихся слов в этом подходе будет тем самым признаком, который мы можем использовать для обучения классификатора. Это не совсем точно, ведь фразы «мне не понравился фильм» и »не мне понравился фильм» отличаются только порядком слов, но имеют разный смысл. Тем не менее, как показывает практика, такого подхода достаточно, чтобы определить тональность или тематику текста.


Определим функцию преобразования текста в «мешок слов», слово будем определять как последовательность русских букв. Будем искать их при помощи регулярных выражений (про регулярные выражения можно почитать тут: https://habr.com/post/349860/). Весь текст будем приводить к нижнему регистру, чтобы слова, написанные в разных регистрах, не отличались друг от друга. 

In [1]:
import re

def text2bow(text):
    result = dict()
    word_regexp = re.compile(r'[а-я]+')
    for word in word_regexp.findall(text.lower()):
        if word not in result:
            result[word] = 1
        else:
            result[word] = result[word] + 1
    return result
            
print(text2bow('Мне не понравился фильм, а ему понравился фильм'))

{'мне': 1, 'не': 1, 'понравился': 2, 'фильм': 2, 'а': 1, 'ему': 1}


Мы возвращаем словарь с частотами слов. Для того, чтобы работать с моделями машинного обучения, нам нужно «табличное» представление даннных. Это можно сделать при помощи pandas, так как в нем уже реализовано преобразование словарей в DataFrame:

In [2]:
import pandas as pd
docs = [
            text2bow('Мне не понравился фильм, а ему понравился фильм'), 
            text2bow('Фильм был очень хороший! Мне понравился'), 
       ]
df = pd.DataFrame(docs)
df

,а,был,ему,мне,не,очень,понравился,фильм,хороший
0,1.0,NaN,1.0,1,1.0,NaN,2,2,NaN
1,NaN,1.0,NaN,1,NaN,1.0,1,1,1.0


Пустоты лучше заполнить нулями:

In [3]:
df = df.fillna(0.0)
df

,а,был,ему,мне,не,очень,понравился,фильм,хороший
0,1.0,0.0,1.0,1,1.0,0.0,2,2,0.0
1,0.0,1.0,0.0,1,0.0,1.0,1,1,1.0


Мы преобразовали тексты в таблицу, теперь к ним можно применять модели машинного обучения. Давайте загрузим настоящие данные — отзывы к фильмам. ОНИ УЖЕ ПРИВЕДЕНЫ К МОДЕЛИ  «мешок слов».

В выборке 30 000 примеров, мы отфильтровали 10 000 самых популярных слов на русском языке, чтобы данных не получилось слишком много. Колонка is_positive содержит целевую функцию (положительный отзыв или отрицательный).

Внимание! Данных довольно много, их загрузка займет продолжительное время.

In [4]:
data = pd.read_csv('train.csv', dtype='uint8')
data.head()

,is_positive,а,абсолютная,абсолютно,абсолютное,абсолютной,абсурд,абсурда,аварии,австралии,...,ярко,яркого,яркое,яркой,яркую,ярости,ярость,ярче,ясно,ящик
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,3,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [5]:
labels = data['is_positive']
del(data['is_positive'])

Разделим выборку на тестовую и контрольную при помощи функции train_test split.

In [6]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = 0.2, random_state = 123)

Обучим модель классификации. Необходимо импортировать модель машинного обучения из библиотеки Sklear, обучить ее и проверить на тестовых данных. Будем строить на основе моделей

- DecisionTreeClassifier 
- RandomForestClassifier 

Чтобы улучшить свой результат, зададим параметры моделей. У модели RandomForestClassfier для улучшения качества можно задавать, например, такие параметры: 
- Количество деревьев(n_estimators)
- Максимальную высоту дерева(max_depth)

Полный список критериев можно посмотреть в оффициальной документации:
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Также для улучшения точности можно использовать логистическую регрессию или градиентный бустинг.

In [7]:
from sklearn.ensemble import RandomForestClassifier

# подставим значения для создания модели
model = RandomForestClassifier(n_estimators=450, min_samples_leaf=10, max_depth=55, random_state=0, criterion = 'gini', bootstrap = True, class_weight = 'balanced')
model.fit(train_data, train_labels)
prediction = model.predict_proba(test_data)[:,1]
# наиболее хороший результат получается при max_depth=55

from sklearn.metrics import roc_auc_score
auc = roc_auc_score(test_labels, prediction)
print("auc:", auc)

auc: 0.9072011679412211


Выполним предсказания для контрольных данных

In [9]:
data_control = pd.read_csv('control.csv')
prediction = model.predict_proba(data_control)[:,1]
print(','.join("{:.3f}".format(p) for p in prediction))

0.628,0.520,0.593,0.791,0.625,0.636,0.680,0.414,0.635,0.546,0.476,0.666,0.504,0.794,0.629,0.690,0.631,0.667,0.490,0.749,0.589,0.589,0.706,0.744,0.690,0.651,0.564,0.516,0.594,0.632,0.755,0.712,0.573,0.775,0.591,0.783,0.717,0.768,0.579,0.668,0.757,0.591,0.767,0.704,0.626,0.566,0.534,0.479,0.794,0.536,0.571,0.706,0.710,0.720,0.735,0.760,0.499,0.637,0.622,0.570,0.633,0.488,0.636,0.684,0.746,0.612,0.789,0.561,0.739,0.650,0.688,0.673,0.623,0.688,0.647,0.694,0.603,0.815,0.582,0.595,0.739,0.458,0.768,0.755,0.668,0.685,0.757,0.539,0.703,0.687,0.744,0.749,0.834,0.716,0.783,0.790,0.703,0.676,0.603,0.774,0.766,0.809,0.731,0.642,0.703,0.620,0.532,0.608,0.609,0.655,0.770,0.605,0.705,0.677,0.654,0.584,0.592,0.374,0.695,0.784,0.806,0.730,0.661,0.730,0.453,0.459,0.545,0.692,0.652,0.644,0.609,0.648,0.809,0.495,0.514,0.682,0.738,0.685,0.536,0.583,0.726,0.503,0.549,0.729,0.726,0.695,0.653,0.569,0.667,0.547,0.708,0.594,0.595,0.820,0.696,0.555,0.729,0.658,0.723,0.661,0.405,0.544,0.725,0.660,0.654,0.681,0.43

# Тестирование

Ниже приведен код, при помощи которого можно проверить, как модель классифицирует реальные образы. Мы преобразуем отзыв в «мешок слов» и подаем на вход модели. 

In [10]:
review = "10/10, фильм на кончиках пальцев!" #напишите сюда все, что думаете


bows = [text2bow(review)]
data_my = pd.DataFrame(bows, columns=train_data.columns).fillna(0.0)
score = model.predict_proba(data_my)[:,1][0]
print("Результат автоматической классификации: {:.3f}".format(score))

Результат автоматической классификации: 0.821


Модель мешка слов, которую мы с вами использовали, является простой. Более сложные модели используют порядок слов, а также приводят слова к нормальному виду. Продвинутые современные методы базируются на рекуррентных и сверточных нейросетях. Смотрите: https://habr.com/company/dca/blog/274027/